In [15]:
import requests
from datetime import datetime
import pandas as pd

### Extract Available Timeslot Response From Smartplay.gov

In [16]:
def fetch_available_venue_timeslot(district_codes, facility_code, play_date):
    
    base_url = "https://www.smartplay.lcsd.gov.hk/rest/facility-catalog/api/v1/publ/facilities"

    params = {
        "distCode": ",".join(district_codes),
        "faCode": facility_code,
        "playDate": play_date
    }

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36 Edg/133.0.0.0',
        'Referer': 'https://www.smartplay.lcsd.gov.hk/facilities/search-result',
        'channel': 'INTERNET'
    }

    response = requests.get(base_url, params=params, headers=headers)
    return response.json()

district_codes=['KC', 'KT', 'SSP', 'WTS', 'YTM']
facility_code='TABT'
play_date='2025-03-05'

available_venue_timeslot = fetch_available_venue_timeslot(district_codes, facility_code, play_date)
available_venue_timeslot

{'code': '0',
 'message': 'success',
 'data': {'morning': {'distList': [{'distCode': 'KC',
     'sessionCount': 33,
     'distName': 'Kowloon City',
     'venueList': [{'venueName': 'Fat Kwong Street Sports Centre',
       'venueId': 223,
       'venueImageUrl': 'https://lcsd-smartplay.oss-cn-hongkong.aliyuncs.com/pic/default_facility.jpeg',
       'sessionCount': 5,
       'fatList': [{'fatName': 'Table Tennis Table (A/C)(Urban)',
         'enFatName': 'Table Tennis Table (A/C)(Urban)',
         'fatId': 509,
         'sessionCount': 5,
         'faCodeName': 'Table Tennis',
         'faCode': 'TABT',
         'faGroupCode': 'BAGM',
         'fvrId': 125005431,
         'sessionList': [{'ssnStartTime': '07:00:00',
           'ssnEndTime': '08:00:00',
           'ssnStartDate': '2025-03-05',
           'available': True,
           'peakHour': False,
           'sfadFlag': False,
           'sessionCount': 2},
          {'ssnStartTime': '08:00:00',
           'ssnEndTime': '09:00:00',


### Data Pre-processing of Available Venue Timeslot

In [17]:
def process_available_venue_timeslot(available_venue_timeslot):
    data = []

    for period in [i for i in available_venue_timeslot["data"].keys() if i != "venueCountList"]:
        # print(available_venue_timeslot["data"][period])

        for district in available_venue_timeslot["data"][period]["distList"]:
            # print(district["distCode"])

            for venue in district["venueList"]:
                # print(" ", venue["venueName"])
                
                for facility in venue["fatList"]:
                    # print("  ", facility["fatName"])

                    for session in facility["sessionList"]:
                        # print("   ", session["ssnStartTime"], session["ssnEndTime"], session["available"])

                        session_info = {"date": session["ssnStartDate"],
                                        "period": period,
                                        "district_code": district["distCode"],
                                        "district_name": district["distName"],
                                        "venue_id": venue["venueId"],
                                        "venue_name": venue["venueName"],
                                        "facility_name": facility["fatName"],
                                        "session_start_time": session["ssnStartTime"],
                                        "session_end_time": session["ssnEndTime"],
                                        "session_available": session["available"]}
                        
                        data.append(session_info)
                        # print(session_info)
    return data

data = process_available_venue_timeslot(available_venue_timeslot)
pd.DataFrame(data)

,date,period,district_code,district_name,venue_id,venue_name,facility_name,session_start_time,session_end_time,session_available
0,2025-03-05,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,Table Tennis Table (A/C)(Urban),07:00:00,08:00:00,True
1,2025-03-05,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,Table Tennis Table (A/C)(Urban),08:00:00,09:00:00,True
2,2025-03-05,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,Table Tennis Table (A/C)(Urban),09:00:00,10:00:00,True
3,2025-03-05,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,Table Tennis Table (A/C)(Urban),10:00:00,11:00:00,False
4,2025-03-05,morning,KC,Kowloon City,223,Fat Kwong Street Sports Centre,Table Tennis Table (A/C)(Urban),11:00:00,12:00:00,False
...,...,...,...,...,...,...,...,...,...,...
999,2025-03-05,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,Table Tennis Table (A/C)(Urban),18:00:00,19:00:00,False
1000,2025-03-05,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,Table Tennis Table (A/C)(Urban),19:00:00,20:00:00,False
1001,2025-03-05,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,Table Tennis Table (A/C)(Urban),20:00:00,21:00:00,False
1002,2025-03-05,evening,YTM,Yau Tsim Mong,70000521,Tai Kok Tsui Sports Centre,Table Tennis Table (A/C)(Urban),21:00:00,22:00:00,True


### Extract District Info Code

In [18]:
def fetch_district_info():
    """Fetches and structures district hierarchy data from the LCSD API."""
    
    # Main response structure
    api_response = {
        "fetch_date": datetime.today().strftime("%Y-%m-%d"),
        "districts": []
    }

    request_headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36",
        "Referer": "https://www.smartplay.lcsd.gov.hk/",
        "Accept": "application/json"
    }

    districts_endpoint = "https://www.smartplay.lcsd.gov.hk/rest/param/api/v1/publ/districts/searching-criteria?pgm=N"

    server_response = requests.get(districts_endpoint, headers=request_headers)
    response_data = server_response.json().get("data", 0)

    if response_data:

        for region_group in response_data:
            region_dict = {
                "region_id": region_group["id"],
                "region_code": region_group["code"].strip(),
                "region_name": region_group["name"].strip(),
                "sc_name": region_group["scName"].strip(),
                "tc_name": region_group["tcName"].strip(),
                "subdistricts": []
            }

            for subdistrict in region_group["children"]:
                subdistrict_dict = {
                    "subdistrict_id": subdistrict["id"],
                    "subdistrict_code": subdistrict["code"].strip(),
                    "subdistrict_name": subdistrict["name"].strip(),
                    "en_name": subdistrict["enName"].strip(),
                    "sc_name": subdistrict["scName"].strip(),
                    "tc_name": subdistrict["tcName"].strip()
                }
                region_dict["subdistricts"].append(subdistrict_dict)

            api_response["districts"].append(region_dict)

        return api_response
    
    return {"status": "error", "code": server_response.status_code}

district_info = fetch_district_info()
district_info

{'fetch_date': '2025-03-03',
 'districts': [{'region_id': 202313440,
   'region_code': 'HK',
   'region_name': 'Hong Kong',
   'sc_name': '香港',
   'tc_name': '香港',
   'subdistricts': [{'subdistrict_id': 202311860,
     'subdistrict_code': 'CW',
     'subdistrict_name': 'Central & Western',
     'en_name': 'Central & Western',
     'sc_name': '中西区',
     'tc_name': '中西區'},
    {'subdistrict_id': 202311861,
     'subdistrict_code': 'EN',
     'subdistrict_name': 'Eastern',
     'en_name': 'Eastern',
     'sc_name': '东区',
     'tc_name': '東區'},
    {'subdistrict_id': 202311869,
     'subdistrict_code': 'SN',
     'subdistrict_name': 'Southern',
     'en_name': 'Southern',
     'sc_name': '南区',
     'tc_name': '南區'},
    {'subdistrict_id': 202311873,
     'subdistrict_code': 'WCH',
     'subdistrict_name': 'Wan Chai',
     'en_name': 'Wan Chai',
     'sc_name': '湾仔区',
     'tc_name': '灣仔區'}]},
  {'region_id': 202313441,
   'region_code': 'KLN',
   'region_name': 'Kowloon',
   'sc_name': '九